# Household Energy Consumption IoT Data Analysis

The UCI Household Energy Consumption dataset contains data on the energy usage of an individual household over a period of four years. This data will be analyzed using PySpark to gain insights into the patterns and trends of energy consumption. In this notebook, we will explore various techniques for analyzing this data to uncover insights that can help households optimize their energy usage.

#### Tableau Dashboard: __[https://public.tableau.com/app/profile/mission.posssible/viz/GlobalPowerConsumptionIoT_16826248629190/Dashboard2?publish=yes](https://public.tableau.com/app/profile/mission.posssible/viz/GlobalPowerConsumptionIoT_16826248629190/Dashboard2?publish=yes)__

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName("HouseholdEnergyConsumption").getOrCreate()

df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("household_power_consumption.csv")

# Convert date and time columns to timestamp format
from pyspark.sql.functions import to_timestamp

df = df.withColumn("Date", to_timestamp("Date", "dd/MM/yyyy"))
df = df.withColumn("Time", to_timestamp("Time", "HH:mm:ss"))

# Convert data types as needed
df = df.withColumn("Global_active_power", df["Global_active_power"].cast("double"))
df = df.withColumn("Global_reactive_power", df["Global_reactive_power"].cast("double"))
df = df.withColumn("Voltage", df["Voltage"].cast("double"))
df = df.withColumn("Global_intensity", df["Global_intensity"].cast("double"))
df = df.withColumn("Sub_metering_1", df["Sub_metering_1"].cast("double"))
df = df.withColumn("Sub_metering_2", df["Sub_metering_2"].cast("double"))
df = df.withColumn("Sub_metering_3", df["Sub_metering_3"].cast("double"))

# Cleaned data to csv file
panda_df = df.toPandas()
panda_df.to_csv('household_power_consumption_cleaned.csv')

In [11]:
# Calculate the average daily energy consumption for each household
daily_energy_df = df.groupby("Date").agg(avg("Global_active_power").alias("Avg_daily_energy_consumption"))

# Generating csv
daily_energy_pd = daily_energy_df.toPandas()
daily_energy_pd.to_csv('daily_energy.csv')

# Analyze the energy consumption patterns based on seasons
seasonal_energy_df = df.withColumn("Season", when(month(col("Date")).between(3, 5), "Spring")
                                             .when(month(col("Date")).between(6, 8), "Summer")
                                             .when(month(col("Date")).between(9, 11), "Fall")
                                             .otherwise("Winter")) \
                             .groupby("Season").agg(avg("Global_active_power").alias("Avg_seasonal_energy_consumption"))

# Generating csv
seasonal_energy_pd = seasonal_energy_df.toPandas()
seasonal_energy_pd.to_csv('seasonal_energy.csv')

# Identify the peak energy usage hours during the day
peak_hours_df = df.groupby(hour("Time")).agg(avg("Global_active_power").alias("Avg_power_consumption")).orderBy(col("Avg_power_consumption").desc())

# Generating csv
peak_hours_pd = peak_hours_df.toPandas()
peak_hours_pd.to_csv('peak_hours.csv')


# Analyze the energy consumption by different appliances
appliance_energy_df = df.agg(sum("Sub_metering_1").alias("Kitchen_energy_consumption"),
                                                         sum("Sub_metering_2").alias("Laundry_energy_consumption"),
                                                         sum("Sub_metering_3").alias("Heating_cooling_energy_consumption"))

# Generating csv
appliance_energy_pd = appliance_energy_df.toPandas()
appliance_energy_pd.to_csv('appliance_energy.csv')

# Analyze the energy consumption patterns based on the day of the week
dayofweek_energy_df = df.withColumn("DayOfWeek", when(dayofweek(col("Date")).isin([2,3,4,5,6]), "Weekday")
                                                          .otherwise("Weekend")) \
                                .groupby("DayOfWeek").agg(avg("Global_active_power").alias("Avg_energy_consumption"))

# Generating csv
dayofweek_energy_pd = dayofweek_energy_df.toPandas()
dayofweek_energy_pd.to_csv('dayofweek_energy.csv')

# Analyze the energy consumption patterns based on the time of day
timeofday_energy_df = df.withColumn("TimeOfDay", when(hour(col("Time")).between(6, 11), "Morning")
                                                           .when(hour(col("Time")).between(12, 17), "Afternoon")
                                                           .when(hour(col("Time")).between(18, 23), "Evening")
                                                           .otherwise("Night")) \
                               .groupby("TimeOfDay").agg(avg("Global_active_power").alias("Avg_energy_consumption"))

# Generating csv
timeofday_energy_pd = timeofday_energy_df.toPandas()
timeofday_energy_pd.to_csv('timeofday_energy.csv')

# Analyze the energy consumption patterns based on the month of the year
monthly_energy_df = df.withColumn("Month", month(col("Date"))) \
                             .groupby("Month").agg(avg("Global_active_power").alias("Avg_monthly_energy_consumption"))

# Generating csv
monthly_energy_pd = monthly_energy_df.toPandas()
monthly_energy_pd.to_csv('monthly_energy.csv')


# Analyze the energy consumption patterns based on different voltage levels
# voltage_energy_df = df.withColumn("VoltageLevel", when(col("Voltage") < 220, "Low")
#                                                           .when(col("Voltage") >= 220 & col("Voltage") < 240, "Medium")
#                                                           .otherwise("High")) \
#                              .groupby("VoltageLevel").agg(avg("Global_active_power").alias("Avg_energy_consumption"))

# # Generating csv
# voltage_energy_pd = voltage_energy_df.toPandas()
# voltage_energy_pd.to_csv('voltage_energy.csv')


# Analyze the energy consumption patterns based on day/night periods
daynight_energy_df = df.withColumn("DayNight", when(hour(col("Time")).between(7, 21), "Day")
                                                         .otherwise("Night")) \
                              .groupby("DayNight").agg(avg("Global_active_power").alias("Average Energy Consumption"))


# Generating csv
daynight_energy_pd = daynight_energy_df.toPandas()
daynight_energy_pd.to_csv('daynight_energy.csv')


+---------+----------------------+
|TimeOfDay|Avg_energy_consumption|
+---------+----------------------+
|  Evening|    1.6038027062585163|
|  Morning|     1.254352484311334|
|Afternoon|     1.066759286417198|
|    Night|    0.5071572293310848|
+---------+----------------------+

None
+------+-------------------------------+
|Season|Avg_seasonal_energy_consumption|
+------+-------------------------------+
|Winter|             1.1082021682977274|
+------+-------------------------------+

None
+--------------------------+--------------------------+----------------------------------+
|Kitchen_energy_consumption|Laundry_energy_consumption|Heating_cooling_energy_consumption|
+--------------------------+--------------------------+----------------------------------+
|                 1229855.0|                 1540831.0|                           6197928|
+--------------------------+--------------------------+----------------------------------+

None
+----------+---------------------+
|hour(T